In [3]:
from datasets import load_dataset

# Load the IMDB dataset 
raw_datasets = load_dataset("imdb")
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define a function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Applying  tokenization
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
# im Selecting a smaller portion of the dataset (15-20%)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["train"]))))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["test"]))))
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model for binary classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Defined the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,                   # Increased learning rate for Task 3
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,                   # Increased number of epochs for Task 3
    weight_decay=0.01,
    logging_steps=100,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,    # 15-20% of dataset for Task 3
    eval_dataset=small_eval_dataset,
)

# Fine-tune the model
trainer.train()



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.337600,0.215331
2,0.177300,0.479398
3,0.078300,0.431788
4,0.033300,0.498305
5,0.014800,0.501820
6,0.000200,0.590601


Epoch,Training Loss,Validation Loss
1,0.337600,0.215331
2,0.177300,0.479398
3,0.078300,0.431788
4,0.033300,0.498305
5,0.014800,0.501820
6,0.000200,0.590601
7,0.004800,0.628097


TrainOutput(global_step=3283, training_loss=0.0978841978183996, metrics={'train_runtime': 3443.9206, 'train_samples_per_second': 7.622, 'train_steps_per_second': 0.953, 'total_flos': 6906665203200000.0, 'train_loss': 0.0978841978183996, 'epoch': 7.0})

In [6]:


results = trainer.evaluate()
print(f"Evaluation results: {results}")
model.save_pretrained("./fine_tuned_bert_binary1")

Evaluation results: {'eval_loss': 0.628097414970398, 'eval_runtime': 115.7867, 'eval_samples_per_second': 32.387, 'eval_steps_per_second': 4.051, 'epoch': 7.0}


In [11]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import numpy as np

# Load the Reuters dataset
dataset = load_dataset("reuters21578", "ModApte")

# Initialize the multilabel binarizer
mlb = MultiLabelBinarizer()

# Fit the multilabel binarizer on the train set
all_labels = [item for sublist in dataset['train']['topics'] for item in sublist]
mlb.fit([all_labels])

# Function to binarize labels
def binarize_labels(example):
    return {'labels': mlb.transform([example['topics']])[0].astype(np.float32)}  # Convert to float32

# Apply binarization to the dataset
dataset = dataset.map(binarize_labels)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare datasets (use 15% of the dataset for this task)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["train"]))))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.15 * len(tokenized_datasets["test"]))))

# Load pre-trained BERT model for multilabel classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(mlb.classes_),
                                                      problem_type="multi_label_classification")

# Define training arguments
# Define training arguments with adjustments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,  # Experiment with this
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,  # Increase the number of epochs
    weight_decay=0.01,
    logging_steps=100,
)


# Define metric function to compute F1 scores
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Apply sigmoid to the raw logits to get probabilities, then threshold at 0.5
    predictions = (torch.sigmoid(torch.Tensor(predictions)) > 0.5).float().numpy()

    # Compute micro and macro F1 scores
    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_bert_multilabel2")


Map:   0%|          | 0/722 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['bfr'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['f-cattle'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['hk'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['sfr'] will be ignored
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/722 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro
1,0.249900,0.053137,0.000000,0.000000
2,0.045100,0.038851,0.000000,0.000000
3,0.036700,0.033392,0.469101,0.008346
4,0.032000,0.029922,0.479889,0.008428
5,0.029400,0.027101,0.484182,0.008432
6,0.026600,0.024492,0.640994,0.015719
7,0.024500,0.022766,0.660976,0.017783
8,0.022900,0.021356,0.696897,0.023896
9,0.022200,0.020867,0.696790,0.026977
10,0.020800,0.020544,0.701422,0.027575


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/di

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation results: {'eval_loss': 0.020544135943055153, 'eval_f1_micro': 0.7014218009478673, 'eval_f1_macro': 0.02757512023031616, 'eval_runtime': 15.1886, 'eval_samples_per_second': 32.524, 'eval_steps_per_second': 4.082, 'epoch': 10.0}


In [2]:

from datasets import load_dataset

# Load your dataset (replace "dataset_name" with the actual dataset you're using)
# For example, let's use 'imdb' as a placeholder
datasets = load_dataset("imdb")

# Now, print the dataset structure
print(datasets)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [1]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
